# How to build a genotype-phenotype map (a.k.a. sequence space) from protein lattice models

This notebook demonstrates how to use Jesse Bloom's **protein lattice model package**, [latticeproteins](), to build genotype-phenotype map. In this case, the phenotypes are estimate from the protein's ability to fold and bind a ligand. You must have `latticeproteins` installed as a dependency for this package. 

We'll begin by importing some of his package here.

In [1]:
import os
from latticeproteins.conformations import *
from latticeproteins.fitness import Fitness

In `latticeproteins`'s `conformations` module, we can build the ensemble of all possible conformations for sequences of the same length.

In [2]:
length = 10
database_dir = "%s/database" % os.getcwd()
c = Conformations(length, database_dir)

In [3]:
from latticegpm.gpm import LatticeGenotypePhenotypeMap
from latticegpm.search import sequence_space

First, we need to find two sequences that have a non-zero fitness and differ at all sites! `search_fitness_landscape` does exactly that.

In [4]:
wildtype, mutant = sequence_space(10, differby=6)
print("Wildtype sequence: " + wildtype)
print("Mutant sequence: " + mutant)

Wildtype sequence: VNAFGNMGSI
Mutant sequence: FNRFGSAFAI


Now, we'll build a sequence space between these two ligands with the `LatticeSequenceSpace` object and print out some example nodes in this space.

In [5]:
# Create an instance of LatticeFitnessSpace
from gpmap.utils import binary_mutations_map
mutations = binary_mutations_map(wildtype, mutant)
sequence_space = LatticeGenotypePhenotypeMap(wildtype, mutations, c)

In [6]:
sequence_space.stabilities[0]

-0.15367685968991651

In [7]:
sequence_space.fracfolded

array([ 0.46165622,  0.46866002,  0.73452499,  0.73629754,  0.70005527,
        0.72489365,  0.72950033,  0.74147926,  0.46531842,  0.47251199,
        0.73604136,  0.73789524,  0.71268092,  0.73995861,  0.7355482 ,
        0.74810991,  0.30156953,  0.30644998,  0.66726161,  0.66858763,
        0.61895162,  0.64221666,  0.67093509,  0.68293781,  0.30599   ,
        0.31119244,  0.66894446,  0.67039782,  0.63407447,  0.66133773,
        0.67681591,  0.68969464,  0.33817964,  0.34208042,  0.68721637,
        0.68797814,  0.6940525 ,  0.70893356,  0.55357275,  0.56117452,
        0.33904918,  0.34329401,  0.68749956,  0.68832623,  0.69706504,
        0.71613331,  0.5552505 ,  0.56371115,  0.25582729,  0.25817047,
        0.66080363,  0.66134319,  0.66549299,  0.67865119,  0.52883303,
        0.53583875,  0.25609311,  0.25883999,  0.66086653,  0.66147833,
        0.66725354,  0.68519342,  0.52910474,  0.53702372])

We can access all sequences and fitness in this space by calling these properties.